![figure](../lab2/lab2_figures/politecnico_h-01.png)

# **Eletrónica Configurável / Configurable Electronics**
#### Mestrado em Engenharia Eletrotécnica / Master in Electrical and Electronic Engineering

## **LabWork3 - Pipelining and Static Timing Analysis**

_____________

## Introduction ##
In this lab you will implement a FIR filter with pipelining. You will analyze the design, perform static timing analysis, implement the design and generate a bitstream. Results will be observed using an Integrated Logic Analyser (ILA).

### Objectives ###
After completing this lab, you will be able to:

* Understand the concept of pipelining and its impact on critical paths 
* Use STA to find the maximum operating frequency of a digital circuit
* Generate and quantize coefficients for digital FIR filters using FdaTool from matlab
* Change the synthesis settings and see their effects on the generated output
* Analyze the post implementation reports with regard to timing performance
* Generate bitstream and verify the functionality in hardware using ILA


In the instructions below **{sources}** refers to `C:\Xilinx\MEE_EC\sources` and **{labs}** refers to `(C:\Xilinx\MEE_EC\labs)`

### Design description ###

The block diagram shown in the following figure shows the structure of the circuit we will design and analyse. It includes several `.vhd` files:
* DDS described in `dds.vhd` and `dds_rom.vhd` files
* FIR Filter described in `fir_filter_N.vhd` and `coef_rom.vhd` files
* Top level design module described in `firN_top.vhd`
* Testbench file given in `firN_top_tb.vhd`

It also provided a constraints file for timing analysis:
* Constraints file is `lab3.xdc`.

![Figure](../lab3/lab3_figures/fig1.png)

______________

## Step 1 - Create a Vivado Design ##

### Step 1.1 ###

In this section you will create a new and empty project for PYNQ-Z2 Board in Vivado. We will use Vivado 2020.1 - note that some options may vary depending on the version you are using.

* Start downloading the source files provided (Moodle) and save them in `{sources}/lab3`.


* Open Vivado and create a new project to start the wizard. Click **Next**.


* Click the browse button of the project location field of the new project form, browse to `c:/Xilinx/MEE_EC/labs` and click **Select**. Enter `lab3` in the project name field. Make sure that the *Create Project Subdirectory* box is checked. Click **Next**.


* We will create an empty RTL project and specify sources later. Select **RTL Project** option in the *Project Type* form, and click **Next**. Make sure that the *Do not specify sources* at this time box is checked.


* In the *Default Part* form, select the *Boards* option and choose the **PYNQ-Z2** board. Click **Next** and then **Finish** to create the Vivado project.


### Step 1.2 ### 

In this section you will add some sources to the project.

* In the *Sources* pane select **Add Sources** by clicking the **plus** sign. Then select *Add or create design sources* and click **Next**.


* Select *Add Files* and browse to the `{sources}\lab3` directory, and select simultaneously all the HDL files except for thetstbench file: `coef_rom.vhd`, `dds_rom.vhd`, `dds.vhd`, `fir_filter_N.vhd` and `firN_top.vdh`. Click **OK** and then **Finish**.


* Repeat the last two steps but now select *Add or create constraints* and click **Next**. Select *Add Files* and browse to the `{sources}\lab3` directory, select `lab3.xdc`, click **OK** and then **Finish**. 


* Repeat the last two steps but now select *Add or create simulation sources* and click **Next**. Select *Add Files* and browse to the `{sources}\lab3` directory, select `firN_top_tb.vhd`, click **OK** and then **Finish**. 

### Step 1.3 ### 

We will now elaborate the design and analyze its architecture.

* In the *Sources* window, open the `fir_filter_N.vhd` file and scroll down to **line 63** and then **line 205**. Compare these lines and observe that this filter has two architecture definitions:
    * `'architecture rtl of fir_filter_N is'` defines a basic RTL FIR filter (line 205)
    * `'architecture rtl_piped of fir_filter_N is'` defines a pipelined RTL FIR filter (line 63)
    

* In the *Sources* window, open the `firN_top.vhd` file and go to **line 107**. Note that we are here instantiating the entity **fir_filter_N** with a basic **rtl** architecture. 


* In the *Flow Navigator*, expand *RTL ANALYSIS* and select **Open Elaborated Design**. Disregard the message click **OK**. The Elaborated Design Shematic canvas will open and should look like the figure. We can see the coefficient memory with parallel outputs, the FIR filter, the DDS sourcing data to the filter and the DDS configuration memory.


* The **DDS** module has an additional debug bus output, which is not represented here for simplicity.


![Figure](../lab3/lab3_figures/fig2.png)


* **Expand** the DDS module in the schematic and observe its structure. we have:
    * 1 input register to store the phase accumulator step (wich will define the sinewave frequency) stored in the external ROM. The ROM has 4 possible outputs wich will allow us to generate 4 different frequencies: 1MHz, 5MHz, 10MHz and 20MHz.
    * 1 register to store the initial sinewave phase, wich has been set to zero in the source file.
    * 1 input register for the synchronous reset which allows us to reset the sinewave phase by forcing the initial phase into the accumulator register, through the multiplexer.
    * 1 phase accumulator, made-up with one adder (`r_nco0_i`) and one register (`r_nco_reg`).
    * 1 Look-up table (`C_LUT_SIN`) with input and output registers.
    * 1 resetable output register.


* **Expand** the FIR filter module in the schematic and observe its structure. we have:
    * 32 memory units in series to implement the serial input parallell output shift register for data;
    * 32 memory units in parallel to register the filters coefficients;
    * 32 multipliers;
    * 5 adder stages in cascade;
    * 1 output register

<div class="alert alert-block alert-info">
<b>Info:</b> The adder module is implemented with 5 cascading stages between the multiplier and the output register. Therefore, we have several combinatorial logic stages that define a very long combinatorial timing path!
</div>


* **Close** the *Elaborated Design* view by clicking on the blue ribbon cross.


*  In the *Sources* window, open the `firN_top.vhd` file. Change **line 104** to `fir_filter_i4: entity work.fir_filter_N(rtl_piped)` and **save** the file.


* **Open Elaborated Design** again, **expand** the FIR filter module in the schematic and observe its structure. Note that registers have been inserted between combinatorial stages - now the design is **pipelined**.


* **Close** the *Elaborated Design* view by clicking on the blue ribbon cross.


* Change **line 104** back to `fir_filter_i4: entity work.fir_filter_N(rtl)` and **save** the file.

_______________

## Step 2 - Simulate the Design ##

### Step 2.1 ### 

In this section you will simulate the design with provided filter coefficients.

* In the *Flow Navigator*, expand *SIMULATION* and select **Run Simulation > Run Behavioral Simulation**. 


* In the *Waveform* window right-click on **sine_out** and choose **Waveform Style > Analog**. 


* Right-click again on **sine_out** and choose **Waveform Style > Analog Settings** and choose the **Hold** interpolation style to be able to see the real generated signal.


* Right-click again on **sine_out** and choose **Radix > Signed Decimal**.


* Repeat the steps described above in order to visualize the real input sinewave **sine_in** in signed decimal representation.


* Restart the simulation using the button **Restart Simulation** and then run it for **20us**. Click on **Zoom to fit** to observe the complete simulation. Note that the filter passes only the 10MHz sinewave and significantly attenuates other frequencies (you are supposed to see even more atenuation in frequencies other than 10MHz).

![Figure](../lab3/lab3_figures/fig7.png)


* Zoom in and out and inspect the waveforms. Note that the maximum amplitude of sine_out when **addr_fcw=2** is **5131**. Note also that the maximum amplitude of sine_out when **addr_fcw=1** is **67**. This corresponds to an attenuation close to 40dB (20log10(5131/67)) when the 5MHz signal is compared to the 10MHz signal (as expected from the fdatool Magniture response plot). 


<div class="alert alert-block alert-info">
<b>Info:</b> If you wish to see more smooth sine waveforms, change the interpolation style back to **linear** in the *Analog Settings* window.
</div>

* **Close** the simulation view.

The filter coefficients stored in the `coef_rom.vhd` were obtained using Matlab's Filter Design & Anlysis Tool (fdatool). The filter was designed as a **Bandpass** FIR filter, order 31, and the specifications shown in the following figure. Reproduce this in Matlab and observe the coefficients. Note that some are positive and some are negative, with a maximum absolute value close to 0,12.

![Figure](../lab3/lab3_figures/fig3.png)

Coefficients were then quantized to fixed-point 8 bits (MSB is the sign, followed by 7 bits fractional), scaled to fully utilize the entire dynamic range. The input data length was set to 14 bits (13 bits fractional) to match our DDS output precision. These configurations are shown in the following figures.

![Figure](../lab3/lab3_figures/fig4.png)

![Figure](../lab3/lab3_figures/fig5.png)

The quantized filter response differs from the original one, but not significantly, as shown the figure below (also provided by fdatool). Observe also the coefficients and note that they are now scalled to fully utilize the representation range (8bits total; 7 bits fractional).

![Figure](../lab3/lab3_figures/fig6.png)

### Step 2.2 ### 

In this section you will learn how to use Matlab fdatool to design filters and generate quantized coefficients

* Open Matlab and type **fdatool** in the *Command Window*. Wait for the fdatool wizard to open.


* Design a **Lowpass FIR** filter with the following specifications. Click **Design Filter** and observe its magnitude response.
    * Design Method: Least-squares
    * Order: 31 (a filter with order N has N+1 coefficients)
    * Frequency Units: MHz
    * Fs: 100
    * Fpass: 7
    * Fstop: 10
    * Wpass: 1
    * Wstop: 10


* Click on the **Set quantization parameters** button on the left-hand side menu and configure:
    * Filter arithmetic: Fixed-point
    * Filter precision: Full
    * Numerator word length: 8
    * Numerator fraction length: 7
    * Scale the numerator coefficients to fully.... box: check
    * Input word length: 14
    * Input fraction length: 13


* Click on **Apply**. The *Magnitude Response* window should exhibit the frequnecy response  of our filter. We can also inspect the coefficients by plotting the impulse response, as shown below.

![Figure](../lab3/lab3_figures/fig8.png)


* To export the coefficients go to **File > Export**, acept the defaults and click **OK**. Note that you have now a new variable in the Matlab's workspace **Num** with the quantized coefficients. You can plot it (`stem(Num)`) to see if it corresponds to the same impulse response shown by fdatool.


* To convert these coeficients to binary format type `num_bin=dec2bin(typecast(int8(Num*2^7),'uint8'))` in the *Command Window*. 


### Step 2.3 ### 

In this section you will modify the filter coefficients and re-simulate the design.

* **Update** the filter coeficients in the `coef_rom.vhd` file with the ones corresponding to this new filter. Click **save**.

* **Run Simulation** again for **20us** and check if it is working as expected. Sinewave signals with 1MHz and 5MHz frequencies should pass the filter while others should be increasingly attenuated as the frequency increases. In your simulation results you are expected to observe higher attenuations that the ones shown in the figure, for the stopbands).

![Figure](../lab3/lab3_figures/fig10.png)

* **Close** the Simulation view when you are satisfied.

_______________

## Step 3 - Perform STA to find Critical Paths and Maximum Clock Frequency ##

### Step 3.1 ### 

On this step you will analyse the synthesized design.

* In the *Sources* window find and open the `lab3.xdc` constraints file. Note that it defines only timing constraints (constraints to I/O locations are commented for now). Note that the clock frequency is defined as 125MHZ (**T=8ns**) with 50% duty-cycle (**[0.000 4.000]**). 


* In the *Flow Navigator*, expand *SYNTHESIS* and select **Run Synthesis** command. In the dialog box accept the defaults and click **OK**. 


* When synthesis finishes choose **Open Synthesized design**.


* In the *Flow Navigator*, open the **Schematic** unde the *SYNTHESIS* menu. Note that IO Buffers have been added to the design. Also design modules have been replaced by FPGA primites. Inside the filter, for example, we have now:
    * Input and coefficient registers replaced by FDCE units (Flip-Flop D with clock enable).
    * Multipliers replaced with DSP48 units.
    * Adders replaced with LUTs and CARRY chains.


* When you are satisfied **close** the schematic.



### Step 3.2 ### 

On this step you will perform STA and find the maximum clock frequency for the circuit with the basic RTL Filter architecture.

* Run **Report Timing Summary**. Accept all defaults and click **OK**.


* Observe the **Timing** tab and note that the design has a Setup **Worst Negative Slack = -7.8ns** which is almost one clock period! Setup **Total Negative Slack = 120.5ns** also tells us that there are many failing paths.
 
 
* Click on the hyperlink of the **Worst Negative Slack**. This allows you to see all failing paths. **Right-click** the worst path and select **sechematic**. Note that this *critical path* is on the filter module, at one of the multiple long combinatorial paths inside the adder unit.


* Reduce the clock frequency on the constraints file `lab3.xdc` to **Tclk=16ns** and 50% duty cycle (**[0.000 8.000]**). Save the file, **close** the synthesized design and **Run Synthesis** again.


* When complete choose to **Open the Synthesized Design** and run **Report Timing Summary** again. Now we can see that there are no negative setup slacks although margins are small. This means we should probably consider this (**62.5MHz**) the maximum circuit clock frequency, but we still have to implement the design. 


<div class="alert alert-block alert-info">
<b>Info:</b> Negative Hold slacks are usually not a problem as hold violations (short paths) are tipically addressed by implementation tools.
</div>


* **Run Implementation** and choose to **Open Implemented Design** when complete. Say **Yes** if prompted to close the Synthesized Design.


* when implementation completes you should receive a message saying that the design failed to meet timing requirements. Click **OK** and observe that hold violations are resolved but there is a small negative setup slack. Indeed margins were small after synthesis! At this point we have two options:
    * Reduce the clock frequency (e.g., to ~55MHz)
    * Try  more aggressive implementation strategy (we will try this one next)
    

* Open the **Design Runs** tab in the bottom canvas, **right-click** on impl_1 and select **Implementation Run Properties**. Inside this window select the **Options** tab and choose a different implementation strategy. Double click **Performance_ExtraTimingOpt** to set it. Click **Yes** to proceed.


* Because implementation became out-of-date you will have to run it again. Right-click on **impl_1** and choose **Launch Run**.


* While waiting for it to complete you can **close** the previous implemented design, as suggested in the yellow ribbon on the top. If you choose not to, you can always reload the new implementation results once it finishes using the hyperlink **Reload** on the ribbon.


* Even with this agressive implementation strategythe design still fails to meet timing so we will have no other option than reducing the clock frequency to ~55MHz (**T=18ns**). Make this change in `lab3.xdc` file and **re-run the design** (both synthesis and implementation). Set the implementation strategy back to **Vivado Implementation Defaults** because it is faster.


* Observe that now the design meets the timing requirements so we found the **maximum clock frequency** (~55MHz) for this design. We could probably push it a little bit higher but not much more (positive slack is below 1ns).

### Step 3.3 ### 

In this section you will perform STA and find the maximum clock frequency for the circuit with the pipelined RTL Filter architecture.

* Open the file `firN_top.vhd` and choose the filter's **pipelined architecture** in **line 104** (rtl_piped) and save the changes.


* Open the file `lab3.xdc` and set the clock frequency back to 125MHz (**T=8ns**) and 50% duty cycle (**[0.000 4.000]**). Set the Virtual clock period also to **8ns**. Save the changes.


* Run **Synthesis** and observe the **Timing** tab (Run *Report Timing Summary*). Note that the design has a small positive setup slack (~1.43ns) and a huge total Hold Negative Slack (-15ns).


* Run **Implementation** and observe the **Timing** tab. After implementation, the design fails to meet timing requirements by ~2ns. This means we have to reduce the clock frequency, but not too much. 
 
 
* Decrease the clock frequency on the constraints file `lab3.xdc` to **Tclk=10ns** and 50% duty cycle (**[0.000 5.000]**), which corresponds to 100MHz. Set the Virtual clock period also to **10ns**. Save the file, and **Run Synthesis and Implementation** again.


* Observe that **timing requirements fail** by a very small amount of time, and in one single path, with this 100MHz clock signal. As this negative slack is very small we will try to use a more agressive implementation strategy. Set it to **Performance_ExtraTimingOpt** and run implementation again.


* Now the design meets the timing requirements, with a positive slack of ~0.12ns, so we can say that the **maximum clock frequency** of this design is 100MHz. Note that this frequency is almost two times faster than the one in the non-pipelined design. This is the impact of pipelining!


* **Close** the implemented design.


_______________

## Step 4 - Synthesize and Implement the Design ##

### Step 4.1 ### 

In this section you will complete the design using the rtl_piped architecture in IP Integrator, running at 100MHz, with an Integrated Logic Analyzer.

* In the *Flow Navigator* select **Create Block Design** and name it **lab3_top**. Click **OK**.


* **Drag** the `firN_top.vhd` source to the design canvas. Note that although the reset input is defined as **active high** in the VHDL code, the module is created with **active low** reset inputs by default. To change this, **double-click** on the **rstb pin** and set the polarity to **ACTIVE HIGH**


* Add one **Clocking Wizard** IP and one **Integrated Logic Analyzer (ILA)** IP from the Xilinx IP core library.


* Make sure the Clocking Wizard is set to output a 100MHz clock from a 125MHz input clock and with **NO Phase Alignment** (Clocking Features: Fequency Synthesis only). If you want you can remove the optional output **locked** because we are not going to use it.


* Double-click the **ILA** and configure the following in the *General Options* tab:
    * Monitor Type: select Native
    * Number of Probes: 4
    * Trigger and Storage Settings: check both *Capture Control* and *Advanced Trigger*
    * Number of Comparators: 2


* Make the necessary connections to have the same design as shown below and change the input port's name accordingly. Click **Validate Design** and **Regenerate Layout**. Click **Save**.


![Figure](../lab3/lab3_figures/fig12.png)


* Open `lab3.xdc` constraints file and update the **ports' names** according to the block design. You will have to:
    * **Update** the clock name to **sys_clock** twice in line 3
    * **Comment** lines 12 and 13 and **uncomment** lines 16 to 18. This updates the clocks to which the reset signals are referenced to. The global **reset** continues referenced to the input sys_clock (125MHz) but the other reset signals (**rstb** and **sync_rst**) have to be referenced to the wizard's output clock (100MHz).
    * **Comment** lines 28 and 29, as these signals will not be FPGA outputs.
    * **Uncomment** lines 35, 38 and 39, 42, 43 and 44, to set the package pinout.
    
    

* Change the input clock period to **8ns**, duty cycle 50%, because the board input clock is 125MHz. Make sure the virtual clock period is **10ns**, because the design internal clock will be 100MHz. **Save** the file.


* In the *Sources* window **right-click** on **lab3_top** and select **Create HDL Wrapper**, letting Vivado manage it and auto-update.


* In the *Sources* window **right-click** on **lab3_top_wrapper** and select **Set as top**. This tells Vivado that this is the design you want to synthesize and implement. Allow this step to complete.


* Select **Generate Bitstream**. This will instruct vivado to run synthesis, implementation and programming steps.



### Step 4.2 ### 

In this section you will run the design and observe the sinewaves in the ILA, as they are being generated and filtered inside the FPGA.

* Make sure the board is powered up and connected to your computer via USB.


* When the Bitstream Generation successfully completes, choose to **Open Hardware Manager** and click **OK**.


* On the **green ribbon** select **Open target > Auto Connect**.


* On the **green ribbon** select **Program device**. Click **Program** and the **ILA** waveform window should open.


* Select the signals on the waveform window and in the **Debug Probe Properties** window find **Display name** and select **Short name**.


* Right click on the sine_in and sine_out signals in the waveform window and configure them as **Analog** style and a **Signed Decimal** representation.


* In the **Settings** tab below the waveform window select:
    * Trigger mode: BASIC_ONLY
    * Capture mode: ALWAYS
    * Trigger position in window: 100
    

* In the menu bar above the waveform window select **Run trigger for this ILA core**. You should be able to see the filter's input and output waveforms at different frequencies (according to the position of switches).


* Our design has a debug port, connected to the **ILA**, which is composed of some internal DDS signals. Open the file `dds.vhd` and check line **107**. This is just to show you how we can observe internal RTL signals using ILA. We will explore the **ILA** trigger and capture settings in the next lab.


__________________

## Homework Challenge ## 

Instatiate four DDS modules in `firN_top`, with different static frequencies (dds_rom is no longer necessary). You can use the same 1MHz, 5MHz, 10MHz and 20MHz frequencies as before. Each DDS will therefore output a sinewave with a different frequency. Add them and truncate the output so that the resulting waveform has the same number of bits expected by the filter's sine input (14 bits). 

Also, change the filter's coefficients in order to have a passband centered in 5MHz. The idea is that the filter should receive a signal with multiple spectral components and pass only the 5MHz component (attenuating all others). **Have fun!**
